In [1]:
# from google.colab import drive
# drive.mount('/content/drive')


In [2]:
!pip install -q faiss-cpu sentence-transformers llama-index llama-index-embeddings-huggingface openai langchain langchain_community pypdf tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently ta

In [3]:
import os

# ✅ Replace with your OpenRouter API key
os.environ["OPENROUTER_API_KEY"] = "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"

# Path to your PDFs in Google Drive
PDFS_PATH = "/content/Maniq/pdfs"

# Path to save FAISS vector DB in Google Drive
VDB_PATH = "/content/Maniq/vector_db"
os.makedirs(VDB_PATH, exist_ok=True)


In [4]:
import glob
from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document as LangchainDocument
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import pickle

class FAISSContextualRAG:
    def __init__(self, vdb_path):
        self.api_key = os.environ.get("OPENROUTER_API_KEY")
        if not self.api_key:
            raise ValueError("OPENROUTER_API_KEY environment variable not set")

        self.client = OpenAI(
            api_key=self.api_key,
            base_url="https://openrouter.ai/api/v1"
        )

        # Embedding model
        self.embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

        # Vectorstore (FAISS)
        self.vdb_path = vdb_path
        self.vectorstore = None

    def load_documents(self, pdf_paths):
        """Load all PDFs into Langchain Documents"""
        pages = []
        for path in pdf_paths:
            loader = PyPDFLoader(path)
            pages.extend(loader.load())
        return [LangchainDocument(page_content=p.page_content, metadata=p.metadata) for p in pages]

    def build_or_load_index(self, docs=None):
        """Build FAISS index if docs provided, else load existing"""
        if docs:
            print("Building new FAISS index...")
            self.vectorstore = FAISS.from_documents(docs, self.embedder)
            self.vectorstore.save_local(self.vdb_path)
            print(f"✅ Vector DB saved to {self.vdb_path}")
        else:
            print("Loading existing FAISS index...")
            self.vectorstore = FAISS.load_local(self.vdb_path, self.embedder, allow_dangerous_deserialization=True)
            print(f"✅ Loaded FAISS DB from {self.vdb_path}")

    def search_for_question(self, question, k=5):
        """Search and answer with context"""
        if not self.vectorstore:
            raise ValueError("Vectorstore is not initialized. Run build_or_load_index first.")

        results = self.vectorstore.similarity_search(question, k=k)
        context = "\n\n".join([doc.page_content for doc in results])
        return self.generate_response(question, context)

    def generate_response(self, question, context):
        prompt = f"""You are a helpful assistant.
You are an expert in Maniq ethnography (ชาติพันธุ์มานิ).
Use the following context to answer the question truthfully.
The answer is thai or english depends on languages of question.
The answer is revised in paragraph, not bullets.
Context:
{context}

Question: {question}
Answer:"""

        completion = self.client.chat.completions.create(
            model="qwen/qwen-2.5-7b-instruct",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.3
        )
# ✅ แสดง token usage
        if hasattr(completion, "usage"):
            print(f"\n🔎 Token usage → Prompt: {completion.usage.prompt_tokens}, "
                  f"Completion: {completion.usage.completion_tokens}, "
                  f"Total: {completion.usage.total_tokens}")

        return {
                    "answer": completion.choices[0].message.content,
                    "usage": {
                        "prompt_tokens": completion.usage.prompt_tokens,
                        "completion_tokens": completion.usage.completion_tokens,
                        "total_tokens": completion.usage.total_tokens
                    }
                }


In [5]:
pdf_paths = glob.glob(f"{PDFS_PATH}/*.pdf")
print(f"Found {len(pdf_paths)} PDFs")

rag = FAISSContextualRAG(vdb_path=VDB_PATH)

if os.path.exists(VDB_PATH) and os.listdir(VDB_PATH):
    # Load existing vector DB
    rag.build_or_load_index()
else:
    # Build new index
    docs = rag.load_documents(pdf_paths)
    print(f"Loaded {len(docs)} chunks from {len(pdf_paths)} PDFs")
    rag.build_or_load_index(docs)


Found 1 PDFs


/tmp/ipython-input-520540078.py:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Loaded 18 chunks from 1 PDFs
Building new FAISS index...
✅ Vector DB saved to /content/Maniq/vector_db


In [6]:
question = "ANYA คืออะไร"
result = rag.search_for_question(question)
print(result["answer"])
print("Token usage:", result["usage"])



🔎 Token usage → Prompt: 5451, Completion: 230, Total: 5681
ANYA คือชุดข้อมูลการอ้างอิงสำหรับการวิเคราะห์ท่าทางที่มีประสิทธิภาพ ซึ่งถูกสร้างขึ้นโดยใช้การเรียนรู้แบบครึ่งติดป้าย (semi-supervised learning) ข้อมูลในชุด ANYA ประกอบด้วยภาพที่ถูกป้ายกำกับโดยผู้เชี่ยวชาญและภาพที่ถูกป้ายกำกับแบบเสมือน (pseudo-labeled) โดยใช้การคลุกคลานแบบ K-means และการป้อนข้อมูลจากผู้เชี่ยวชาญ ชุดข้อมูล ANYA ช่วยปรับปรุงคุณภาพของโมเดลโดยเพิ่มจำนวนตัวอย่างที่ใช้ในการฝึกฝน ทำให้โมเดลสามารถสืบสานและปรับตัวได้ดีในสภาพแวดล้อมที่ไม่เคยเห็นมาก่อน
Token usage: {'prompt_tokens': 5451, 'completion_tokens': 230, 'total_tokens': 5681}


In [7]:
%%time

question = "Who are maniq"
answer = rag.search_for_question(question)
print("\nQ:", question)
print("A:", answer)



🔎 Token usage → Prompt: 8241, Completion: 106, Total: 8347

Q: Who are maniq
A: {'answer': 'Maniq, also known as Mien or Mien people, are a Tai-Kadai ethnic group primarily found in northern Thailand, southern China, and northern Vietnam. They are known for their distinctive culture, language, and traditional practices. Maniq communities have their own language, which is part of the Tai-Kadai language family, and they maintain strong social and cultural ties within their communities. Their traditional way of life includes agriculture, particularly rice farming, and they are recognized for their skilled craftsmanship in textiles and metalwork.', 'usage': {'prompt_tokens': 8241, 'completion_tokens': 106, 'total_tokens': 8347}}
CPU times: user 32.8 ms, sys: 2.07 ms, total: 34.9 ms
Wall time: 2.93 s


In [8]:
import gradio as gr
import time

def ask_mani(question):
    """Function to call the RAG model and return enriched answer info."""

    start_cpu = time.process_time()
    start_wall = time.time()

    # เรียก RAG
    result = rag.search_for_question(question)

    end_cpu = time.process_time()
    end_wall = time.time()

    # ถ้า rag.search_for_question คืนแค่ string → ต้องแก้ให้ generate_response return dict
    if isinstance(result, dict):
        answer = result["answer"]
        usage = result.get("usage", {})
    else:
        # fallback กรณียังเป็น string
        answer = result
        usage = {"prompt_tokens": "?", "completion_tokens": "?", "total_tokens": "?"}

    # สร้างข้อความสวยงาม
    response = f"""### 🧾 คำตอบ
{answer}

---

### 📊 ข้อมูลการประมวลผล
- ⏱ CPU time: {end_cpu - start_cpu:.2f} วินาที
- ⏱ Wall time: {end_wall - start_wall:.2f} วินาที
- 🔎 Token usage → Prompt: {usage.get('prompt_tokens', '?')}, Completion: {usage.get('completion_tokens', '?')}, Total: {usage.get('total_tokens', '?')}
- 📏 ความยาวคำถาม: {len(question)} ตัวอักษร
- 📏 ความยาวคำตอบ: {len(answer)} ตัวอักษร
"""

    return response


# Create the Gradio interface
iface = gr.Interface(
    fn=ask_mani,
    inputs=gr.Textbox(label="Enter your question about the Maniq people:"),
    outputs=gr.Markdown(),  # ✅ เปลี่ยนเป็น Markdown เพื่อ render สวย
    title="Maniq Knowledge Base (RAG)",
    description="Ask questions about the Maniq people based on the provided documents."
)

# Launch the interface
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d7e6566fc64e160256.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# ทดสอบความแม่นยำ

In [ ]:
import time

def test_rag_performance(questions):
    """
    Tests the performance of the RAG model with a list of questions.

    Args:
        questions (list): A list of strings, where each string is a question.

    Returns:
        list: A list of dictionaries, where each dictionary contains the
              answer and performance metrics for a question.
    """
    results = []
    for question in questions:
        start_cpu = time.process_time()
        start_wall = time.time()

        result = rag.search_for_question(question)

        end_cpu = time.process_time()
        end_wall = time.time()

        if isinstance(result, dict):
            answer = result["answer"]
            usage = result.get("usage", {})
        else:
            answer = result
            usage = {"prompt_tokens": "?", "completion_tokens": "?", "total_tokens": "?"}

        results.append({
            "question": question,
            "answer": answer,
            "cpu_time": end_cpu - start_cpu,
            "wall_time": end_wall - start_wall,
            "token_usage": usage,
            "question_length": len(question),
            "answer_length": len(answer)
        })
    return results



In [ ]:
# Example usage:
test_questions = ["มานิคืออะไร", "เบาหวานคืออะไร"]
performance_results = test_rag_performance(test_questions)
for res in performance_results:
    print(res)


🔎 Token usage → Prompt: 3043, Completion: 191, Total: 3234

🔎 Token usage → Prompt: 4484, Completion: 128, Total: 4612
{'question': 'มานิคืออะไร', 'answer': 'มานิคือกลุ่มชนพื้นเมืองในภาคใต้ของไทย ที่อาศัยอยู่ในผืนป่าบนเทือกเขาบรรทัด บริเวณจังหวัดตรัง พัทลุง สตูล และเทือกเขาสันกาลาคีรีในเขตจังหวัดยะลา และนราธิวาส. พวกเขาเรียกตัวเองว่า "มานิ" แปลว่าคน บ่งบอกถึงความเป็นมนุษย์อย่างเท่าเทียมและมีศักดิ์ศรี. พวกเขาเป็นกลุ่มชนในวัฒนธรรมการหาของป่าล่าสัตว์ และดำรงชีวิตอยู่กับธรรมชาติอย่างกลมกลืน.', 'cpu_time': 0.05051150900000323, 'wall_time': 3.0388576984405518, 'token_usage': {'prompt_tokens': 3043, 'completion_tokens': 191, 'total_tokens': 3234}, 'question_length': 11, 'answer_length': 334}
{'question': 'เบาหวานคืออะไร', 'answer': 'ข้อมูลในบริบทที่ให้มาไม่มีการกล่าวถึงโรคเบาหวาน ดังนั้นไม่มีข้อมูลที่สามารถตอบคำถามเกี่ยวกับการอธิบายถึงโรคเบาหวานได้จากบริบทนี้ ถ้าต้องการข้อมูลเกี่ยวกับโรคเบาหวาน ควรให้บริบทที่เกี่ยวข้องกับเรื่องนี้หรือหาข้อมูลเพิ่มเติมจากแหล่งที่น่าเชื่อถือ', 'cpu_time': 0.03

In [ ]:
import pandas as pd

# Assuming performance_results is already populated from running test_rag_performance
# Example usage of test_rag_performance (uncomment if needed)
test_questions = ["มานิคืออะไร", "เบาหวานคืออะไร"]
performance_results = test_rag_performance(test_questions)

# Convert the results to a pandas DataFrame
df_results = pd.DataFrame(performance_results)

# Extract token usage into separate columns
df_results['prompt_tokens'] = df_results['token_usage'].apply(lambda x: x.get('prompt_tokens', '?'))
df_results['completion_tokens'] = df_results['token_usage'].apply(lambda x: x.get('completion_tokens', '?'))
df_results['total_tokens'] = df_results['token_usage'].apply(lambda x: x.get('total_tokens', '?'))

# Drop the original token_usage column
df_results = df_results.drop(columns=['token_usage'])


# Specify the path to save the CSV file in Google Drive
csv_save_path = "/content/performance_results.csv"

# Save the DataFrame to a CSV file
df_results.to_csv(csv_save_path, index=False)

print(f"✅ Performance results saved to {csv_save_path}")


🔎 Token usage → Prompt: 3043, Completion: 158, Total: 3201

🔎 Token usage → Prompt: 4484, Completion: 97, Total: 4581
✅ Performance results saved to /content/performance_results.csv


In [ ]:
# ================================
# ManiqRAG Test Harness (Colab-ready)
# ================================
import os, time, pandas as pd

# --- ตั้งค่าพาธไฟล์ทดสอบ (คอลัมน์ที่ 2 คือคำถาม) ---
# ตัวอย่าง: ถ้าคุณอัปโหลดไฟล์ไปที่ /content/questions-answer.xlsx บน Colab
TEST_FILE_PATH = "/content/questions-answer.xlsx"   # <-- แก้ให้ตรงกับพาธจริงของคุณ

# --- สมมติว่าคุณมีออบเจ็กต์ rag จากคลาส FAISSContextualRAG อยู่แล้ว ---
# ถ้าในโน้ตบุ๊กยังไม่มี ให้ uncomment/ปรับให้ตรงโปรเจ็กต์เดิมของคุณ:
# from your_module import FAISSContextualRAG  # ถ้าคุณแยกเป็นโมดูล
# VDB_PATH = "/content/drive/MyDrive/Research/Maniq/vector_2_db"
# rag = FAISSContextualRAG(vdb_path=VDB_PATH)
# rag.build_or_load_index()  # โหลด FAISS ที่สร้างไว้แล้ว

def _call_rag(question: str):
    """
    เรียก rag.search_for_question(question) แล้วคืนผลลัพธ์มาตรฐาน
    รองรับทั้งกรณีที่ฟังก์ชันคืน dict (มี 'answer' และ 'usage') หรือคืน string
    """
    start_cpu = time.process_time()
    start_wall = time.time()
    try:
        result = rag.search_for_question(question)
    except Exception as e:
        end_cpu = time.process_time()
        end_wall = time.time()
        return {
            "question": question,
            "answer": f"[ERROR] {e}",
            "token_usage": {"prompt_tokens":"?", "completion_tokens":"?", "total_tokens":"?"},
            "cpu_time_sec": round(end_cpu - start_cpu, 4),
            "wall_time_sec": round(end_wall - start_wall, 4),
            "answer_length": 0,
            "question_length": len(question),
            "status": "error"
        }

    end_cpu = time.process_time()
    end_wall = time.time()

    # ทำให้เป็นรูปแบบมาตรฐาน
    if isinstance(result, dict):
        answer = result.get("answer", "")
        usage = result.get("usage", {}) or result.get("token_usage", {})
        token_usage = {
            "prompt_tokens": usage.get("prompt_tokens", "?"),
            "completion_tokens": usage.get("completion_tokens", "?"),
            "total_tokens": usage.get("total_tokens", "?"),
        }
    else:
        answer = str(result)
        token_usage = {"prompt_tokens":"?", "completion_tokens":"?", "total_tokens":"?"}

    return {
        "question": question,
        "answer": answer,
        "token_usage": token_usage,
        "cpu_time_sec": round(end_cpu - start_cpu, 4),
        "wall_time_sec": round(end_wall - start_wall, 4),
        "answer_length": len(answer),
        "question_length": len(question),
        "status": "ok"
    }

def test_rag_performance(questions, sleep_sec: float = 0.0):
    """
    รันคำถามเป็นลิสต์ ผ่าน RAG แล้วคืน list ของ dict
    - questions: list[str]
    - sleep_sec: เว้นช่วงระหว่างคำถาม (กัน rate limit) ถ้าไม่จำเป็นให้ = 0
    """
    results = []
    for q in questions:
        q = str(q).strip()
        if not q:
            continue
        rec = _call_rag(q)
        results.append(rec)
        if sleep_sec > 0:
            time.sleep(sleep_sec)
    return results

# ===== 1) โหลดไฟล์เทสต์ แล้วดึง "คอลัมน์ที่ 2" เป็นคำถาม =====
if not os.path.exists(TEST_FILE_PATH):
    raise FileNotFoundError(f"ไม่พบไฟล์ทดสอบ: {TEST_FILE_PATH} (โปรดแก้ TEST_FILE_PATH ให้ถูกต้อง)")

# พยายามอ่านแบบมีส่วนหัวก่อน ถ้าไม่เจอคอลัมน์ที่ 2 จะ fallback เป็น header=None
try:
    df_test = pd.read_excel(TEST_FILE_PATH, engine="openpyxl")
except Exception:
    df_test = pd.read_excel(TEST_FILE_PATH, header=None, engine="openpyxl")

# ดึงคอลัมน์ที่ 2 (index=1) เป็นชุดคำถาม
if df_test.shape[1] < 2:
    raise ValueError("ไฟล์ทดสอบต้องมีอย่างน้อย 2 คอลัมน์ (คอลัมน์ที่ 2 เป็นคำถาม)")

test_questions = (
    df_test.iloc[:, 1]  # คอลัมน์ที่สอง (index=1)
    .dropna()
    .astype(str)
    .map(str.strip)
    .tolist()
)

print(f"พบคำถามสำหรับทดสอบ {len(test_questions)} ข้อ (จากคอลัมน์ที่ 2)")

# ===== 2) รันทดสอบ RAG =====
performance_results = test_rag_performance(test_questions)

# ===== 3) สร้าง DataFrame และแตกคอลัมน์ token_usage =====
df_results = pd.DataFrame(performance_results)
df_results["prompt_tokens"] = df_results["token_usage"].apply(lambda x: x.get("prompt_tokens", "?"))
df_results["completion_tokens"] = df_results["token_usage"].apply(lambda x: x.get("completion_tokens", "?"))
df_results["total_tokens"] = df_results["token_usage"].apply(lambda x: x.get("total_tokens", "?"))
df_results = df_results.drop(columns=["token_usage"])

# ===== 4) บันทึกผลเป็น CSV =====
csv_save_path = "/content/performance_results.csv"
df_results.to_csv(csv_save_path, index=False, encoding="utf-8-sig")
print(f"✅ Performance results saved to {csv_save_path}")

# (ทางเลือก) แสดงตัวอย่าง 5 แถวแรก
df_results.head()


พบคำถามสำหรับทดสอบ 89 ข้อ (จากคอลัมน์ที่ 2)

🔎 Token usage → Prompt: 4853, Completion: 309, Total: 5162

🔎 Token usage → Prompt: 4779, Completion: 327, Total: 5106

🔎 Token usage → Prompt: 3817, Completion: 378, Total: 4195

🔎 Token usage → Prompt: 3284, Completion: 500, Total: 3784

🔎 Token usage → Prompt: 4286, Completion: 500, Total: 4786

🔎 Token usage → Prompt: 3195, Completion: 500, Total: 3695

🔎 Token usage → Prompt: 4387, Completion: 500, Total: 4887

🔎 Token usage → Prompt: 4691, Completion: 330, Total: 5021

🔎 Token usage → Prompt: 3538, Completion: 354, Total: 3892

🔎 Token usage → Prompt: 4129, Completion: 500, Total: 4629

🔎 Token usage → Prompt: 4895, Completion: 245, Total: 5140

🔎 Token usage → Prompt: 3295, Completion: 500, Total: 3795

🔎 Token usage → Prompt: 4418, Completion: 120, Total: 4538

🔎 Token usage → Prompt: 3895, Completion: 500, Total: 4395

🔎 Token usage → Prompt: 4277, Completion: 500, Total: 4777

🔎 Token usage → Prompt: 3668, Completion: 231, Total: 3

,question,answer,cpu_time_sec,wall_time_sec,answer_length,question_length,status,prompt_tokens,completion_tokens,total_tokens
0,เขาถ่ายทอดความรู้การอยู่กับป่าให้เด็กๆ อย่างไร?,ข้อความในบริบทไม่ได้กล่าวถึงวิธีการถ่ายทอดความ...,0.0746,4.4440,520,47,ok,4853,309,5162
1,นักวิจัยระบุตัวตนทางภาษาของชุมชนอย่างไร?,นักวิจัยระบุตัวตนทางภาษาของชุมชนว่า:\n\n1. ในเ...,0.0520,3.3377,563,40,ok,4779,327,5106
2,งานจำแนกภาษามานิในกลุ่มอัสเลียนเหนืออย่างไร?,งานจำแนกภาษามานิในกลุ่มอัสเลียนเหนือได้รับการศ...,0.0576,4.8456,674,44,ok,3817,378,4195
3,เอกสารอธิบายความแตกต่างชื่อภาษาตามพื้นที่อย่างไร?,เอกสารอธิบายความแตกต่างชื่อภาษาตามพื้นที่ดังนี...,0.0927,10.5304,906,49,ok,3284,500,3784
4,งานแสดงพัฒนาการรูปแบบการตั้งถิ่นฐานอย่างไร?,งานแสดงพัฒนาการรูปแบบการตั้งถิ่นฐานของชาติพันธ...,0.0693,6.7311,824,43,ok,4286,500,4786
